In [5]:
import sys, os, time
import numpy as np
from datetime import datetime
from joblib import Parallel, delayed
import logging

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from estimators.statistical_descriptor import Nagler_WS
# from plot.figure_roc import ROC_plot
from utils.dataset_management import parse_pipeline
from utils.dataset_load import shuffle_data, DatasetLoader
from utils.fold_management import FoldManagement
from utils.label_management import LabelManagement
from utils.balance_management import *
from utils.figures import plot_boxplots, plot_roc_curves
from utils.files_management import (
    load_yaml,
    dump_pkl,
    init_logger,
    report_prediction,
    report_metric_from_log,
    set_folder,
    logger_dataset,
    logger_fold,
    save_metrics
)

In [13]:
x = {'commands': ["python evaluation/learning_models.py --data_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5 --fold_method mFold --labeling_method 3labels --balancing_method undersample --request (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) --shuffle_data True --balance_data True --import_list from sklearn.svm import SVC --import_list from sklearn.neighbors import KNeighborsClassifier --import_list from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier --import_list from sklearn.linear_model import LogisticRegression --import_list from sklearn.neural_network import MLPClassifier --import_list from estimators.statistical_descriptor import * --import_list from estimators.band_transform import * --pipeline [['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]] --pipeline [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]] --pipeline [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]] --pipeline [['SVMrbf_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['SVC', {'kernel': 'rbf'}, {'probability': True}]] --pipeline [['LogisticR_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['LogisticRegression']] --pipeline [['AdaBoost_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['AdaBoostClassifier', {'n_estimators': 200}]] --metrics_to_report f1_score_macro --metrics_to_report f1_score_weighted --metrics_to_report f1_score_multiclass --metrics_to_report accuracy_score --metrics_to_report precision_score_macro --metrics_to_report recall_score_macro --metrics_to_report roc_auc_score --metrics_to_report log_loss --metrics_to_report kappa_score --metrics_to_report confusion_matrix --seed 100150 --storage_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_15/group_0", "python evaluation/learning_models.py --data_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5 --fold_method mFold --labeling_method 3labels --balancing_method undersample --request (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) --shuffle_data True --balance_data True --import_list from sklearn.svm import SVC --import_list from sklearn.neighbors import KNeighborsClassifier --import_list from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier --import_list from sklearn.linear_model import LogisticRegression --import_list from sklearn.neural_network import MLPClassifier --import_list from estimators.statistical_descriptor import * --import_list from estimators.band_transform import * --pipeline [['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]] --pipeline [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]] --pipeline [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]] --pipeline [['SVMrbf_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['SVC', {'kernel': 'rbf'}, {'probability': True}]] --pipeline [['LogisticR_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['LogisticRegression']] --pipeline [['AdaBoost_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['AdaBoostClassifier', {'n_estimators': 200}]] --metrics_to_report f1_score_macro --metrics_to_report f1_score_weighted --metrics_to_report f1_score_multiclass --metrics_to_report accuracy_score --metrics_to_report precision_score_macro --metrics_to_report recall_score_macro --metrics_to_report roc_auc_score --metrics_to_report log_loss --metrics_to_report kappa_score --metrics_to_report confusion_matrix --seed 42 --storage_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_15/group_1"], 'commit_sha': 'c81ee8e1343c100c727255126fefe579905078f7', 'end_time': [(2024, 6, 28, 15, 17, 17, 264199), (2024, 6, 28, 15, 19, 26, 635923)], 'executable': 'evaluation/learning_models.py', 'executable_command': 'python', 'experiment_id': 1, 'groups_of_parameters': [{'--balance_data': True, '--balancing_method': 'undersample', '--data_path': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5', '--fold_method': 'mFold', '--import_list': ['from sklearn.svm import SVC', 'from sklearn.neighbors import KNeighborsClassifier', 'from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier', 'from sklearn.linear_model import LogisticRegression', 'from sklearn.neural_network import MLPClassifier', 'from estimators.statistical_descriptor import *', 'from estimators.band_transform import *'], '--labeling_method': '3labels', '--metrics_to_report': ['f1_score_macro', 'f1_score_weighted', 'f1_score_multiclass', 'accuracy_score', 'precision_score_macro', 'recall_score_macro', 'roc_auc_score', 'log_loss', 'kappa_score', 'confusion_matrix'], '--pipeline': [[['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]], [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]], [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]], [['SVMrbf_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['SVC', {'kernel': 'rbf'}, {'probability': True}]], [['LogisticR_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['LogisticRegression']], [['AdaBoost_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['AdaBoostClassifier', {'n_estimators': 200}]]], '--request': '(date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000))', '--seed': 100150, '--shuffle_data': True}, {'--balance_data': True, '--balancing_method': 'undersample', '--data_path': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5', '--fold_method': 'mFold', '--import_list': ['from sklearn.svm import SVC', 'from sklearn.neighbors import KNeighborsClassifier', 'from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier', 'from sklearn.linear_model import LogisticRegression', 'from sklearn.neural_network import MLPClassifier', 'from estimators.statistical_descriptor import *', 'from estimators.band_transform import *'], '--labeling_method': '3labels', '--metrics_to_report': ['f1_score_macro', 'f1_score_weighted', 'f1_score_multiclass', 'accuracy_score', 'precision_score_macro', 'recall_score_macro', 'roc_auc_score', 'log_loss', 'kappa_score', 'confusion_matrix'], '--pipeline': [[['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]], [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]], [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]], [['SVMrbf_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['SVC', {'kernel': 'rbf'}, {'probability': True}]], [['LogisticR_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['LogisticRegression']], [['AdaBoost_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['AdaBoostClassifier', {'n_estimators': 200}]]], '--request': '(date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000))', '--seed': 42, '--shuffle_data': True}], 'main_pid': 19631, 'pids': ['19646', '20779'], 'repertories': ['results/pipeline/run_15/group_0', 'results/pipeline/run_15/group_1'], 'run_id': 15, 'start_time': [(2024, 6, 28, 15, 15, 4, 929685), (2024, 6, 28, 15, 17, 17, 313866)], 'status': ['finished', 'finished'], 'storage_path': 'results/pipeline/run_15', 'working_directory': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline'}

In [39]:
[x["groups_of_parameters"][0]["--pipeline"][i][0][0] for i in range(len(x["groups_of_parameters"][0]["--pipeline"]))]

['KNN_direct',
 'RandomForest_direct',
 'MLP_direct',
 'SVMrbf_direct',
 'LogisticR_direct',
 'AdaBoost_direct']

In [37]:
[x["groups_of_parameters"][i]["--pipeline"][k][0][0] for k in range(len(x["groups_of_parameters"][0]["--pipeline"])) for i in range(len(x["groups_of_parameters"]))]

2